In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [7]:
word_index = imdb.get_word_index()
reverse_word_index = {value + 3: key for key, value in word_index.items()}
reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNK>"
reverse_word_index[3] = "<UNUSED>"

In [5]:
# load the model
model = load_model('imdb_model.h5')
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_7 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
# check the weights after training
model.get_weights()

[array([[-1.20748416e-01, -7.28926420e-01,  3.18270456e-03, ...,
          2.79931538e-02,  1.23245656e+00, -5.91899991e-01],
        [-5.80407567e-02, -8.15063789e-02, -6.24142326e-02, ...,
         -5.91492765e-02,  7.94320181e-02, -4.96901013e-02],
        [-1.72529131e-01,  6.33738935e-04,  3.51754613e-02, ...,
         -1.33068368e-01, -1.25024483e-01, -1.56050295e-01],
        ...,
        [-1.29213789e-02,  3.62613685e-02,  8.33230559e-03, ...,
          6.20999597e-02, -5.48201054e-02,  5.80748804e-02],
        [-1.21802427e-02,  8.29470828e-02, -6.23348914e-03, ...,
         -1.85300931e-02,  6.47094622e-02,  5.05927540e-02],
        [ 1.03615426e-01, -2.05473080e-02, -1.72111318e-01, ...,
          1.58408303e-02,  4.79948409e-02,  5.30931279e-02]],
       shape=(10000, 128), dtype=float32),
 array([[ 0.04070063,  0.16883475, -0.05790351, ..., -0.05266754,
          0.00916256, -0.00406296],
        [ 0.05603652, -0.15614708,  0.0569495 , ..., -0.09686481,
         -0.0771924

In [12]:
# to decode review
def decode_review(encoded_review):
    return " ".join([reverse_word_index.get(i,'?') for i in encoded_review])

# to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word,2) for word in words]   # if word not present in worddata of imdb replace with unk
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [13]:
# prediction
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [ ]:
# test the model with user input
example_review1 = "This movie was fantastic! The acting was great and the plot was thrilling."
prediction = predict_sentiment(example_review)
print(f'Sentiment :{prediction[0]} Score:{prediction[1]}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Sentiment :Positive Score:0.8115929961204529


In [26]:
a = ' The movie was boring, slow, badly acted and a complete waste of time.'
predict = model.predict(preprocess_text(a))
predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


array([[0.31924734]], dtype=float32)

In [25]:
"bad" in word_index, "movie" in word_index


(True, True)